# Toronto Neighborhoods Assignment

# PART I

### First import necessary libraries (as per NYC example)

In [107]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import web-scraping library BS4 and requests
from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Scrape website, store table data in text file, create dataframe with table contents and clean dataframe

In [108]:
# scrape website and store data in html format
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")
page_content

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":867606113,"wgRevisionId":867606113,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","

In [110]:
# select only the table data (list of neighborhoods) and store in text format
# notice from the html file that table entries have the <tr> class, so we select using this criteria
table_data = []
for i in range(0, len(page_content.find_all("tr"))):
    table_entries = page_content.find_all("tr")[i].text
    table_data.append(table_entries)

table_data

['\nPostcode\nBorough\nNeighbourhood\n',
 '\nM1A\nNot assigned\nNot assigned\n',
 '\nM2A\nNot assigned\nNot assigned\n',
 '\nM3A\nNorth York\nParkwoods\n',
 '\nM4A\nNorth York\nVictoria Village\n',
 '\nM5A\nDowntown Toronto\nHarbourfront\n',
 '\nM5A\nDowntown Toronto\nRegent Park\n',
 '\nM6A\nNorth York\nLawrence Heights\n',
 '\nM6A\nNorth York\nLawrence Manor\n',
 "\nM7A\nQueen's Park\nNot assigned\n",
 '\nM8A\nNot assigned\nNot assigned\n',
 '\nM9A\nEtobicoke\nIslington Avenue\n',
 '\nM1B\nScarborough\nRouge\n',
 '\nM1B\nScarborough\nMalvern\n',
 '\nM2B\nNot assigned\nNot assigned\n',
 '\nM3B\nNorth York\nDon Mills North\n',
 '\nM4B\nEast York\nWoodbine Gardens\n',
 '\nM4B\nEast York\nParkview Hill\n',
 '\nM5B\nDowntown Toronto\nRyerson\n',
 '\nM5B\nDowntown Toronto\nGarden District\n',
 '\nM6B\nNorth York\nGlencairn\n',
 '\nM7B\nNot assigned\nNot assigned\n',
 '\nM8B\nNot assigned\nNot assigned\n',
 '\nM9B\nEtobicoke\nCloverdale\n',
 '\nM9B\nEtobicoke\nIslington\n',
 '\nM9B\nEtobico

##### Store table data into a dataframe and clean it up

In [113]:
# firstly create dataframe, removing last 5 rows which are in excess
raw_table = pd.DataFrame(table_data).iloc[:-5,:]
raw_table

,0
0,\nPostcode\nBorough\nNeighbourhood\n
1,\nM1A\nNot assigned\nNot assigned\n
2,\nM2A\nNot assigned\nNot assigned\n
3,\nM3A\nNorth York\nParkwoods\n
4,\nM4A\nNorth York\nVictoria Village\n
5,\nM5A\nDowntown Toronto\nHarbourfront\n
6,\nM5A\nDowntown Toronto\nRegent Park\n
7,\nM6A\nNorth York\nLawrence Heights\n
8,\nM6A\nNorth York\nLawrence Manor\n
9,\nM7A\nQueen's Park\nNot assigned\n


In [119]:
# secondly split string in each row using '\n' separator, removing excess columns
# assign first row as column names and remove it afterwards
split_table = pd.DataFrame(raw_table[0].str.split('\n',-1).tolist()).iloc[:,1:-1]
split_table.columns = split_table.iloc[0,:]
split_table = split_table.iloc[1:,:]
split_table

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [120]:
# thirdly clean up the dataframe
# if Borough not assigned remove row entry
# if Neighborhood not assigned replace with name of Borough
cleaned_table = split_table

cleaned_table = cleaned_table[cleaned_table.Borough != 'Not assigned'].reset_index(drop=True)

for i in range(0,cleaned_table.shape[0]):
    if cleaned_table['Neighbourhood'][i] =='Not assigned':
        cleaned_table['Neighbourhood'][i] = cleaned_table['Borough'][i]
        
cleaned_table

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [190]:
# fourthly group by the same postcode
grouped_obj = cleaned_table.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_table = pd.DataFrame(grouped_obj).reset_index()

grouped_table

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [303]:
grouped_table.shape

(103, 3)

# PART II

### Get geo-coordinates

##### The below code should get the coordinates, but it is often timed out

In [ ]:
import geocoder # import geocoder
latitude = pd.DataFrame(np.zeros(grouped_table.shape[0]),columns=['Latitude'])
longitude = pd.DataFrame(np.zeros(grouped_table.shape[0]),columns=['Longitude'])

for i in range(0,grouped_table.shape[0])
    temp = None
    # loop until you get the coordinates
    while(temp is None):
        gcd = geocoder.google('{}, Toronto, Ontario'.format(grouped_table['Postcode'][i]))
        temp = gcd.latlng

    latitude.iloc[i,] = temp[0]
    longitude.iloc[i,] = temp[1]

all_in = pd.concat([grouped_table,latitude,longitude], axis=1)

##### Instead we use the file provided

In [209]:
# import the file
gc_data = pd.read_csv('Geospatial_Coordinates.csv')
gc_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [212]:
# rename postcode column to match across dataframes
gc_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)
gc_data

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [213]:
# join dataframes
merged_table = grouped_table.merge(gc_data, on="Postcode", how='left')
merged_table

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [214]:
print('The dataframe has {} boroughs.'.format(len(merged_table['Borough'].unique())))

The dataframe has 11 boroughs.


### Create a map of Toronto with neighborhoods superimposed on top.

In [216]:
# create map of Toronto using latitude and longitude values
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_table['Latitude'], merged_table['Longitude'], merged_table['Borough'], merged_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/Users/alicechen/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


# PART III

### Run analysis on all boroughs

##### Define Foursquare Credentials and Version (cell removed from submission)

##### Run analysis for one borough as example

In [239]:
# get the top 100 venues that are in one postcode within a radius of 700 meters.

borough_latitude = merged_table.loc[0, 'Latitude'] # borough latitude value
borough_longitude = merged_table.loc[0, 'Longitude'] # borough longitude value

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 700 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=D5BJJXC4DKDRYYL1440P4ZCZ1ROKA3N0GJO4KNTLRN5U3EGB&client_secret=1P0HJLEZFAVKTPU0YPNWSSGX2CKNI0TOKTLU1UNMGM4VDMVH&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=700&limit=100'

In [240]:
# build the request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0da5c81ed2194a74d0c51d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d669cba83865481c948fa53-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'primary': True,
         'shortName': 'Spa'}],
       'id': '4d669cba83865481c948fa53',
       'location': {'address': '8130 Sheppard Ave E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside Ave',
        'distance': 595,
        'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
         'Toronto ON M1B 3W3',
         'Canada'],
        'labeledLatLngs': [{'label': 'd

In [241]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [242]:
# clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Tim Hortons,Coffee Shop,43.802000,-79.198169
4,Lee Valley,Hobby Shop,43.803161,-79.199681


In [243]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [265]:
# create a function to repeat the same process to all the postcodes

def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [268]:
# run analysis for all postcodes
toronto_venues_PC = getNearbyVenues(names=merged_table['Postcode'],
                                   latitudes=merged_table['Latitude'],
                                   longitudes=merged_table['Longitude']
                                  )
print(toronto_venues_PC.shape)
toronto_venues_PC.head()

(3682, 7)


,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
3,M1B,43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
4,M1B,43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store


In [269]:
toronto_venues_PC.groupby('Postcode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M1B,9,9,9,9,9,9
M1C,5,5,5,5,5,5
M1E,9,9,9,9,9,9
M1G,4,4,4,4,4,4
M1H,21,21,21,21,21,21
M1J,8,8,8,8,8,8
M1K,21,21,21,21,21,21
M1L,15,15,15,15,15,15
M1M,4,4,4,4,4,4


In [253]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 319 uniques categories.


##### Analyse each postcode

In [254]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move borough column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spor

In [271]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
toronto_onehot['Postcode'] = toronto_venues_PC['Postcode'] 

# move postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spo

In [272]:
toronto_onehot.shape

(3682, 320)

In [273]:
# roup rows by postcode and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spo

In [274]:
toronto_grouped.shape

(102, 320)

In [275]:
num_top_venues = 5

for postcode in toronto_grouped['Postcode']:
    print("----"+postcode+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == postcode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                           venue  freq
0           Fast Food Restaurant  0.22
1              Martial Arts Dojo  0.11
2  Paper / Office Supplies Store  0.11
3                    Bus Station  0.11
4                            Spa  0.11


----M1C----
                        venue  freq
0              Breakfast Spot   0.4
1                         Bar   0.2
2                Burger Joint   0.2
3          Italian Restaurant   0.2
4  Modern European Restaurant   0.0


----M1E----
                  venue  freq
0  Fast Food Restaurant  0.22
1            Sports Bar  0.11
2           Coffee Shop  0.11
3           Pizza Place  0.11
4            Beer Store  0.11


----M1G----
              venue  freq
0       Coffee Shop  0.50
1              Park  0.25
2  Business Service  0.25
3             Plaza  0.00
4      Noodle House  0.00


----M1H----
               venue  freq
0        Coffee Shop  0.10
1             Bakery  0.10
2  Indian Restaurant  0.10
3        Yoga Studio  0.05
4   Asian R

In [276]:
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [277]:
# create the new dataframe and display the top 10 venues for each postcode
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Spa,Martial Arts Dojo,Coffee Shop,African Restaurant,Hobby Shop,Bus Station,Paper / Office Supplies Store,Doner Restaurant,Dessert Shop
1,M1C,Breakfast Spot,Italian Restaurant,Burger Joint,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run
2,M1E,Fast Food Restaurant,Coffee Shop,Greek Restaurant,Sports Bar,Fried Chicken Joint,Beer Store,Grocery Store,Pizza Place,College Gym,Doner Restaurant
3,M1G,Coffee Shop,Business Service,Park,Ethiopian Restaurant,Empanada Restaurant,Event Space,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner
4,M1H,Indian Restaurant,Bakery,Coffee Shop,Yoga Studio,Caribbean Restaurant,Athletics & Sports,Bank,Burger Joint,Bus Line,Chinese Restaurant
5,M1J,Fast Food Restaurant,Pizza Place,Restaurant,Convenience Store,Sandwich Place,Coffee Shop,College Rec Center,Dog Run,Design Studio,Dessert Shop
6,M1K,Coffee Shop,Discount Store,Convenience Store,Train Station,Metro Station,Department Store,Sandwich Place,Bus Station,Bus Line,Rental Car Location
7,M1L,Coffee Shop,Bus Line,Diner,Bakery,Park,Bus Station,Convenience Store,Intersection,Fast Food Restaurant,Soccer Field
8,M1M,Furniture / Home Store,Wings Joint,Pizza Place,Burger Joint,Dive Bar,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant
9,M1N,Bank,College Stadium,Diner,Café,Park,Discount Store,Thai Restaurant,Skating Rink,General Entertainment,Dog Run


### Run Cluster Analysis

In [280]:
# Run k-means to cluster the postcodes into 5 clusters.
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add cluster lables to the dataframe
toronto_grouped_clustering

array([0, 0, 4, 2, 0, 4, 0, 0, 4, 1], dtype=int32)

In [282]:
len(kmeans.labels_)

102

In [296]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each
toronto_merged = postcode_venues_sorted

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with merged_table to add latitude/longitude for each borough
toronto_merged = toronto_merged.join(merged_table.set_index('Postcode'), on='Postcode')

# drop rows with #NA
toronto_merged.dropna(axis=0)

toronto_merged.head() # check the last columns!

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Fast Food Restaurant,Spa,Martial Arts Dojo,Coffee Shop,African Restaurant,Hobby Shop,Bus Station,Paper / Office Supplies Store,Doner Restaurant,Dessert Shop,0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Breakfast Spot,Italian Restaurant,Burger Joint,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,0,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Fast Food Restaurant,Coffee Shop,Greek Restaurant,Sports Bar,Fried Chicken Joint,Beer Store,Grocery Store,Pizza Place,College Gym,Doner Restaurant,4,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Coffee Shop,Business Service,Park,Ethiopian Restaurant,Empanada Restaurant,Event Space,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,2,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Indian Restaurant,Bakery,Coffee Shop,Yoga Studio,Caribbean Restaurant,Athletics & Sports,Bank,Burger Joint,Bus Line,Chinese Restaurant,0,Scarborough,Cedarbrae,43.773136,-79.239476


##### Visualize the resulting clusters

In [297]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Examine Clusters

In [298]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude
0,Fast Food Restaurant,African Restaurant,Hobby Shop,Bus Station,Paper / Office Supplies Store,Doner Restaurant,Dessert Shop,0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Breakfast Spot,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,0,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
4,Indian Restaurant,Caribbean Restaurant,Athletics & Sports,Bank,Burger Joint,Bus Line,Chinese Restaurant,0,Scarborough,Cedarbrae,43.773136,-79.239476
6,Coffee Shop,Metro Station,Department Store,Sandwich Place,Bus Station,Bus Line,Rental Car Location,0,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,Coffee Shop,Park,Bus Station,Convenience Store,Intersection,Fast Food Restaurant,Soccer Field,0,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
10,Indian Restaurant,Light Rail Station,Furniture / Home Store,Thrift / Vintage Store,Fast Food Restaurant,Chinese Restaurant,Latin American Restaurant,0,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
12,American Restaurant,Sushi Restaurant,Shopping Mall,Supermarket,Breakfast Spot,Motorcycle Shop,Pool Hall,0,Scarborough,Agincourt,43.794200,-79.262029
17,Clothing Store,Shopping Mall,Theater,Sporting Goods Shop,Food Court,Tea Room,Toy / Game Store,0,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
18,Bank,Café,Chinese Restaurant,Restaurant,Skating Rink,Shopping Mall,Dessert Shop,0,North York,Bayview Village,43.786947,-79.385975
21,Coffee Shop,Ramen Restaurant,Fast Food Restaurant,Sushi Restaurant,Sandwich Place,Korean Restaurant,Dessert Shop,0,North York,Willowdale South,43.770120,-79.408493


In [299]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude
9,Bank,Park,Discount Store,Thai Restaurant,Skating Rink,General Entertainment,Dog Run,1,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
20,Coffee Shop,Indian Restaurant,Liquor Store,Bus Line,Park,Sandwich Place,Bank,1,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
22,Park,Intersection,Dog Run,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,1,North York,York Mills West,43.752758,-79.400049
23,Pizza Place,Yoga Studio,Dog Run,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,1,North York,Willowdale West,43.782736,-79.442259
24,Furniture / Home Store,Park,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,1,North York,Parkwoods,43.753259,-79.329656
30,Pizza Place,Park,Bank,Dog Run,Design Studio,Dessert Shop,Dim Sum Restaurant,1,North York,Downsview West,43.739015,-79.506944
33,Coffee Shop,Park,Playground,Electronics Store,Dive Bar,Department Store,Design Studio,1,North York,Victoria Village,43.725882,-79.315572
35,Beer Store,Intersection,Athletics & Sports,Asian Restaurant,Spa,Video Store,Park,1,East York,Woodbine Heights,43.695344,-79.318389
43,Coffee Shop,Empanada Restaurant,Design Studio,Dessert Shop,Event Space,Dim Sum Restaurant,Diner,1,Central Toronto,Lawrence Park,43.728020,-79.388790
47,Grocery Store,Candy Store,Bank,Japanese Restaurant,Sandwich Place,Tennis Court,Café,1,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [300]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude
3,Coffee Shop,Empanada Restaurant,Event Space,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,2,Scarborough,Woburn,43.770992,-79.216917
29,Coffee Shop,Dive Bar,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,2,North York,"CFB Toronto, Downsview East",43.737473,-79.464763


In [301]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude
19,Pool,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,3,North York,"Silver Hills, York Mills",43.75749,-79.374714


In [302]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude
2,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Grocery Store,Pizza Place,College Gym,Doner Restaurant,4,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
5,Fast Food Restaurant,Sandwich Place,Coffee Shop,College Rec Center,Dog Run,Design Studio,Dessert Shop,4,Scarborough,Scarborough Village,43.744734,-79.239476
8,Furniture / Home Store,Dive Bar,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,4,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
11,Pizza Place,Convenience Store,Rental Car Location,Restaurant,Seafood Restaurant,Burger Joint,Breakfast Spot,4,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
13,Pharmacy,Sandwich Place,Thai Restaurant,Seafood Restaurant,Fried Chicken Joint,Noodle House,Intersection,4,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
14,Pizza Place,Korean Restaurant,Caribbean Restaurant,Bubble Tea Shop,Fast Food Restaurant,Fried Chicken Joint,BBQ Joint,4,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
15,Fast Food Restaurant,Sandwich Place,Nail Salon,Japanese Restaurant,Other Great Outdoors,Auto Garage,Grocery Store,4,Scarborough,"L'Amoreaux West, Steeles West",43.799525,-79.318389
16,Pharmacy,Bakery,Bank,Diner,Recreation Center,Shopping Mall,Sandwich Place,4,North York,Hillcrest Village,43.803762,-79.363452
27,Pizza Place,Sandwich Place,Sushi Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Bank,Restaurant,4,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
28,Coffee Shop,Japanese Restaurant,Bar,Pizza Place,Fast Food Restaurant,Electronics Store,Dive Bar,4,North York,"Northwood Park, York University",43.767980,-79.487262
